В качестве лабораторной работы по теме "Ансамбли моделей" предлагается участие в [соревановании на Kaggle](https://www.kaggle.com/competitions/ensembles-competition/leaderboard).

Основные тезисы и правила



1.   В этом семестре обязательным требованием является размещение результатов выполнения лабораторных работ (всех, начиная с текущей) в своем репозитории на github. Формат — блокнот .ipynb, содержащий программный код и необходимые пояснения markdown.
2.   Топ-3 команды, которые рассказывали о своей работе, получают автоматически 5 по этой лабе (при выполнении пункта 1. можно общий блокнот, если он один на всех, но в своем репозитории с кратким описанием своего вклада в общую работу).
3. Остальные участники размещают также свои материалы у себя в репозитории. В случае командной работы тоже описать свой вклад (конкретно вы, что делали в рамках проекта).
4. Без защиты можно автоматически получить оценку 3. Для более высокой оценки необходимо защитить свою работу. Защита в виде беседы, где вы опять же расскажате о том, как добились результата и о своем вкладе в случае командной работы.





In [41]:
import pandas as pd
import json
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.decomposition import PCA

import numpy as np
from catboost import CatBoostRegressor
# from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor

from sklearn.metrics import mean_absolute_error
import optuna

from catboost import CatBoostClassifier, Pool

In [9]:
X_train = pd.read_csv('train_contest.csv')
X_test = pd.read_csv('for_prediction.csv')
y_train = X_train['mean_salary']
X_train = X_train.drop(columns=['mean_salary'])

X_train.head()

,premium,name,department,has_test,response_letter_required,area,type,address,response_url,sort_point_distance,...,working_days,working_time_intervals,working_time_modes,accept_temporary,description,experience,key_skills,specializations,region,immediate_redirect_url
0,False,Агент по недвижимости,NaN,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Санкт-Петербург', 'street': 'Невский...",NaN,NaN,...,[],[],[],False,"<p>Мы ищем людей, которым не достаточно того, ...","{'id': 'noExperience', 'name': 'Нет опыта'}","[{'name': 'Жилая недвижимость'}, {'name': 'Акт...","[{'id': '20.20', 'name': 'Агент', 'profarea_id...",Санкт-Петербург,NaN
1,False,IT-специалист/ техник,NaN,False,False,"{'id': '1130', 'name': 'Братск', 'url': 'https...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,NaN,...,[],[],[],False,<ul> </ul> <p><em><strong>Навыки</strong></em>...,"{'id': 'between1And3', 'name': 'От 1 года до 3...",[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Иркутская область,NaN
2,False,Старший/ведущий инженер-программист,NaN,False,False,"{'id': '76', 'name': 'Ростов-на-Дону', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Ростов-на-Дону', 'street': '60К-9, 1...",NaN,NaN,...,[],[],[],False,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Linux'}, {'name': 'Bash'}, {'name':...","[{'id': '1.221', 'name': 'Программирование, Ра...",Ростовская область,NaN
3,False,Эксперт / методист приемной комиссии,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': '1-я Миусская ули...",NaN,NaN,...,[],[],[],False,<p>Департамент информационных технологий РХТУ ...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': '1С программирование'}, {'name': 'Об...","[{'id': '3.150', 'name': 'Менеджмент продукта ...",Москва,NaN
4,False,Табельщица/табельщик,NaN,False,False,"{'id': '1613', 'name': 'Энгельс', 'url': 'http...","{'id': 'open', 'name': 'Открытая'}","{'city': None, 'street': None, 'building': Non...",NaN,NaN,...,[],[],[],False,<p><strong>Обязанности:</strong></p> <ul> <li>...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Составление отчетности'}, {'name': ...","[{'id': '18.142', 'name': 'Машиностроение', 'p...",Саратовская область,NaN


## Подготовка данных

In [10]:
def dataframe_convert_bool_to_int(dataframe):
    bool_columns = dataframe.select_dtypes(include=['bool']).columns
    for col in bool_columns:
        dataframe[col] = dataframe[col].astype(int)

    return dataframe


def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    return text.strip()


def combine_dict_values(x, key):
    if pd.isna(x) or x == '[]':
        return None
    try:
        dict_list = ast.literal_eval(x)
        return ','.join([d[key] for d in dict_list])
    except:
        return None


def convert_to_lowercase(dataframe: pd.DataFrame) -> pd.DataFrame:
    text_columns = dataframe.select_dtypes(include=['object', 'category']).columns

    for column in text_columns:
        dataframe[column] = dataframe[column].apply(lambda x: x.lower() if isinstance(x, str) else x)

    return dataframe

In [11]:
# def add_vector_embeddings(dataframe: pd.DataFrame):
#     text_columns = dataframe.select_dtypes(include=['object', 'category']).columns

#     for column in text_columns[:]:
#         print(column)
#         dataframe[column+'_embedding'] = dataframe[column].apply(lambda x: model.encode(x))
#     print('embeddings added')
#     return dataframe

In [39]:
def preprocess_data(salary, labels_to_drop=None):
    # Удаление не нужных признаков
    columns_to_drop_null = ["response_url", "sort_point_distance", "insider_interview", "relations", "working_days", "working_time_intervals", "working_time_modes", "department", 'address'] # address, department и area - вопрос как лучше с ними или без
    columns_to_drop_unused = ["alternate_url", "url", "immediate_redirect_url", "contacts", "published_at", "created_at"]
    salary = salary.drop(columns=columns_to_drop_null+columns_to_drop_unused)
    salary = dataframe_convert_bool_to_int(salary)

    # Достаем из json формата данные
    salary['area'] = salary['area'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['id']
    )
    salary['type'] = salary['type'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['id']
    )

    salary['employer'] = salary['employer'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['name']
    )
    salary['schedule'] = salary['schedule'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['id']
    )
    salary['experience'] = salary['experience'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['name']
    )

    salary['key_skills'] = salary['key_skills'].apply(combine_dict_values, key='name')
    salary['specialization_name'] = salary['specializations'].apply(combine_dict_values, key='name')
    salary['specialization_profarea_name'] = salary['specializations'].apply(combine_dict_values, key='profarea_name')
    salary['snippet'] = salary['snippet'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['requirement']
    )
    salary.drop(columns=['specializations'], inplace=True)

    salary["description"] = salary["description"].apply(preprocess_text)
    salary.fillna('empty', inplace=True)

    salary = convert_to_lowercase(salary)
    if labels_to_drop is not None:
        salary = salary.drop(columns=labels_to_drop)
        salary = pd.get_dummies(salary)

    cat_cols = salary.select_dtypes(include=['object']).columns.tolist()
    salary_encoded = pd.get_dummies(salary, columns=cat_cols)

    for col in cat_cols:
        salary[col] = salary[col].apply(lambda x: x.split(',')[0])

    return salary

## Optuna

In [116]:
X_train_prep = preprocess_data(X_train)
X_test_prep = preprocess_data(X_test)
X_train_prep.head()

,premium,name,has_test,response_letter_required,area,type,archived,employer,snippet,schedule,accept_temporary,description,experience,key_skills,region,specialization_name,specialization_profarea_name
0,0,агент по недвижимости,0,0,2,open,0,арин,уверенное знание компьютера‚ хорошая навигация...,flexible,0,мы ищем людей которым не достаточно того что у...,нет опыта,жилая недвижимость,санкт-петербург,агент,строительство
1,0,it-специалист/ техник,0,0,1130,open,0,смп-38,навыки. коммуникативные навыки для общения с п...,flyinflyout,0,навыки коммуникативные навыки для общения с по...,от 1 года до 3 лет,empty,иркутская область,начальный уровень,информационные технологии
2,0,старший/ведущий инженер-программист,0,0,76,open,0,сармат,высшее инженерно-техническое образование. знан...,fullday,0,обязанности разработка приложений под windows ...,от 1 года до 3 лет,linux,ростовская область,программирование,информационные технологии
3,0,эксперт / методист приемной комиссии,0,0,1,open,0,рхту им. д.и. менделеева,высшее образование. опыт работы с 1с университ...,fullday,0,департамент информационных технологий рхту име...,от 1 года до 3 лет,1с программирование,москва,менеджмент продукта (product manager),маркетинг
4,0,табельщица/табельщик,0,0,1613,open,0,сигнал,знание в совершенстве программ 1с,fullday,0,обязанности учёт рабочего времени сотрудников ...,от 1 года до 3 лет,составление отчетности,саратовская область,машиностроение,производство


In [118]:
X_test_prep.head()

,Id,premium,name,has_test,response_letter_required,area,type,archived,employer,snippet,schedule,accept_temporary,description,experience,key_skills,region,specialization_name,specialization_profarea_name
0,0,0,помощник маркетолога,0,0,1,open,0,eko bike,empty,remote,0,в itкомпанию ищем интернетмаркетолога удаленно...,от 1 года до 3 лет,контекстная реклама,москва,печатная реклама,маркетинг
1,1,0,менеджер по персоналу,0,0,2,open,0,ваш дом,высшее образование.,fullday,0,мы приглашаем на работу соискателей на должнос...,нет опыта,подбор персонала,санкт-петербург,рекрутмент,управление персоналом
2,2,0,"инструктор на детскую игровую площадку (тц ""ме...",0,0,99,open,0,мегаполис,бесконечная любовь к детям. творческий подход ...,fullday,0,если ты любишь играть прыгать и бегать являешь...,нет опыта,empty,республика башкортостан,тренерский состав,спортивные клубы
3,3,0,программист delphi,0,0,4,open,0,ams software,хорошее знание и опыт работы в delphi 2010 и в...,remote,0,компания ams software разработчик популярных ...,от 3 до 6 лет,delphi,новосибирская область,программирование,информационные технологии
4,4,0,ведущий специалист вкс,0,0,1,open,0,технокад,высшее техническое образование. продвинутый по...,fullday,0,компания quotтехнокадquot один из крупнейших ...,от 1 года до 3 лет,настройка пк,москва,начальный уровень,информационные технологии


In [79]:
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_train_prep, y_train, test_size=0.2, random_state=42)
cat_cols = X_train_cat.select_dtypes(include=['object']).columns.tolist()
y_train_cat = y_train_cat.apply(pd.to_numeric, errors='coerce')
y_test_cat = y_test_cat.apply(pd.to_numeric, errors='coerce')
y_train_cat.head()

2351      50000.0
12428    421400.0
26550     28000.0
15214     25000.0
26465    240000.0
Name: mean_salary, dtype: float64

In [136]:
model = CatBoostRegressor(
    iterations=500,      # Количество итераций (деревьев)
    learning_rate=0.1,   # Скорость обучения
    depth=6,            # Глубина деревьев
    loss_function='RMSE',  # Функция потерь (можно также 'MAE', 'Quantile')
    verbose=100,         # Вывод лога каждые 100 итераций
    cat_features=cat_cols
)

model.fit(X_train_cat, y_train_cat, eval_set=(X_test_cat, y_test_cat))

0:	learn: 102965.8019520	test: 94152.7914774	best: 94152.7914774 (0)	total: 11ms	remaining: 5.51s
100:	learn: 72147.8454756	test: 65979.2957672	best: 65979.2957672 (100)	total: 906ms	remaining: 3.58s
200:	learn: 67275.7927835	test: 64634.1716191	best: 64551.1336493 (162)	total: 1.83s	remaining: 2.72s
300:	learn: 64654.6380012	test: 63800.7036936	best: 63780.5581925 (295)	total: 2.84s	remaining: 1.88s
400:	learn: 62115.9555350	test: 63349.7995808	best: 63152.3927656 (352)	total: 3.85s	remaining: 950ms
499:	learn: 60380.9848229	test: 63140.7007084	best: 63140.7007084 (499)	total: 4.83s	remaining: 0us

bestTest = 63140.70071
bestIteration = 499



In [107]:
def objective(trial, X_train, y_train, X_test, y_test, cat_cols):
    train_pool = Pool(X_train, y_train, cat_features=cat_cols)
    test_pool = Pool(X_test, y_test, cat_features=cat_cols)

    param = {
        # Основные параметры
        'iterations': trial.suggest_int('iterations', 100, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'depth': trial.suggest_int('depth', 2, 10),  # оптимальная глубина обычно 4-8
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        
        # Параметры для работы с категориальными признаками
        'one_hot_max_size': trial.suggest_int('one_hot_max_size', 2, 25),
        'has_time': trial.suggest_categorical('has_time', [False]),  # для статических данных
        
        # Регуляризация
        'random_strength': trial.suggest_float('random_strength', 1e-8, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 10.0),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 15),
        
        # Контроль переобучения
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 10, 100),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        
        # Конфигурация обучения
        'loss_function': 'RMSE',
        'verbose': False,
        'random_seed': 42,
        'cat_features': cat_cols
    }

    model = CatBoostRegressor(**param)
    model.fit(train_pool, eval_set=test_pool)
    y_pred = model.predict(X_test)
    
    return mean_absolute_error(y_test, y_pred)

In [108]:
study = optuna.create_study(direction='minimize')
study.optimize(lambda x: objective(x, X_train=X_train_cat, X_test=X_test_cat, y_test=y_test_cat, y_train=y_train_cat, cat_cols=cat_cols), n_trials=10000, timeout=60*3)

[I 2025-06-04 01:18:32,310] A new study created in memory with name: no-name-96faffbf-92b7-4417-9541-982f0ce8cabe
[I 2025-06-04 01:18:38,299] Trial 0 finished with value: 34117.90541479713 and parameters: {'iterations': 686, 'learning_rate': 0.008660555051256553, 'depth': 8, 'l2_leaf_reg': 3.1064616961937587e-07, 'one_hot_max_size': 21, 'has_time': False, 'random_strength': 1.7693737278705205e-07, 'bagging_temperature': 2.0202607511812607, 'leaf_estimation_iterations': 5, 'early_stopping_rounds': 26, 'min_data_in_leaf': 88}. Best is trial 0 with value: 34117.90541479713.
[I 2025-06-04 01:18:44,041] Trial 1 finished with value: 32190.832678307004 and parameters: {'iterations': 1433, 'learning_rate': 0.0686850630720028, 'depth': 7, 'l2_leaf_reg': 6.246814636511708e-06, 'one_hot_max_size': 13, 'has_time': False, 'random_strength': 1.5460902361893865, 'bagging_temperature': 6.718515377309435, 'leaf_estimation_iterations': 9, 'early_stopping_rounds': 76, 'min_data_in_leaf': 10}. Best is tri

In [111]:
param

{'iterations': 364,
 'learning_rate': 0.05299653140296253,
 'depth': 9,
 'l2_leaf_reg': 6.1882900110378615e-06,
 'one_hot_max_size': 16,
 'has_time': False,
 'random_strength': 1.481626423131064e-07,
 'bagging_temperature': 3.3476906417409027,
 'leaf_estimation_iterations': 2,
 'early_stopping_rounds': 88,
 'min_data_in_leaf': 74}

In [129]:
param = study.best_params
model2 = CatBoostRegressor(**param, cat_features=cat_cols)
model2.fit(X_train_prep, y_train)

0:	learn: 102319.5969686	total: 19.1ms	remaining: 6.92s
1:	learn: 100252.1089746	total: 39.4ms	remaining: 7.13s
2:	learn: 98309.9089767	total: 59.6ms	remaining: 7.17s
3:	learn: 96493.0483470	total: 71ms	remaining: 6.39s
4:	learn: 94799.0236244	total: 82.7ms	remaining: 5.94s
5:	learn: 93303.0638858	total: 95.5ms	remaining: 5.7s
6:	learn: 91813.1252693	total: 111ms	remaining: 5.67s
7:	learn: 90585.6029542	total: 131ms	remaining: 5.81s
8:	learn: 89376.9892153	total: 144ms	remaining: 5.69s
9:	learn: 88017.2308565	total: 156ms	remaining: 5.54s
10:	learn: 86764.4864170	total: 173ms	remaining: 5.56s
11:	learn: 85638.3401908	total: 187ms	remaining: 5.48s
12:	learn: 84550.6836330	total: 201ms	remaining: 5.42s
13:	learn: 83622.7399766	total: 219ms	remaining: 5.46s
14:	learn: 82713.6766998	total: 238ms	remaining: 5.53s
15:	learn: 81889.8188241	total: 255ms	remaining: 5.55s
16:	learn: 81156.7468171	total: 266ms	remaining: 5.42s
17:	learn: 80471.9838459	total: 279ms	remaining: 5.36s
18:	learn: 7992

In [143]:
y_pred_optuna = model2.predict(X_test_prep[X_train_cat.columns])

## Подготовка предсказания

In [144]:
%%capture
!warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

X_test_prep = preprocess_data(X_test)

missing_cols = set(X_train_prep.columns) - set(X_test_prep.columns)
for col in missing_cols:
    X_test_prep[col] = 0

X_test_prep = X_test_prep[X_train_prep.columns]

In [145]:
def pred_to_csv(y_pred, name_file):
    df_result = pd.DataFrame(data={"Predicted":list(y_pred)})
    df_result = df_result.reset_index()
    df_result = df_result.rename(columns={"index":"Id"})
    df_result.to_csv(f"{name_file}.csv", index=False)

In [146]:
pred_to_csv(y_pred_optuna, "submit.csv")

## Выводы:

1. **Наше решение:** предобработка признаков, признаки в эмбединги + обучение модели LGBMRegressor с подбором параметров с помощью Optuna
2. **Наша команда:** Андрей, Женя, Гоша, Марат. Андрей, Женя - подготовка, отбор и очистка фич, Марат - формирование эмбедингов, тестирование альтернативных подходов, Гоша - тестирование и сравнение различных моделей + настройка Optuna.
3. **В итоге:** 4 место, немного не хватило до 3.